In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
year = 2024
all_matches = []

In [3]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [4]:
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)

In [5]:
standings_table = soup.select('table.stats_table')[0]
links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

In [6]:
team_urls

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/s

In [7]:
team_urls[:1]

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats']

In [8]:
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text)
    # Scores & Fixtures
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    matches = matches[matches["Comp"] == "Premier League"]
    matches["Season"] = f"{year-1}-{year}"
    matches["Team"] = team_name
    matches["url"] = f"https://fbref.com{links[0]}"
    all_matches.append(matches)
    time.sleep(1)
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("season23_24.csv", index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_21156\3202361865.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_21156\3202361865.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_21156\3202361865.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_21156\3202361865.py:6: FutureWarning: Passing lite

In [9]:
matches_df = pd.read_csv("season23_24.csv")

In [11]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          760 non-null    object 
 1   time          760 non-null    object 
 2   comp          760 non-null    object 
 3   round         760 non-null    object 
 4   day           760 non-null    object 
 5   venue         760 non-null    object 
 6   result        760 non-null    object 
 7   gf            760 non-null    int64  
 8   ga            760 non-null    int64  
 9   opponent      760 non-null    object 
 10  xg            760 non-null    float64
 11  xga           760 non-null    float64
 12  poss          760 non-null    float64
 13  attendance    754 non-null    float64
 14  captain       760 non-null    object 
 15  formation     760 non-null    object 
 16  referee       760 non-null    object 
 17  match report  760 non-null    object 
 18  notes         0 non-null      

#### Chia bớt data tập train qua bên tập test

##### 1. Cho đội Arsenal

In [12]:
arsenal_train = pd.read_csv("../feature_engineering_data/train/arsenal.csv")
arsenal_test = pd.read_csv("../feature_engineering_data/test/arsenal.csv")

In [13]:
arsenal_train.shape, arsenal_test.shape

((875, 25), (38, 25))

In [14]:
moved_season = arsenal_train[(arsenal_train["season"] == "2021-2022") | (arsenal_train["season"] == "2022-2023")]
moved_season.shape

(76, 25)

In [15]:
# Thêm các mùa giải vào tập test
arsenal_test = pd.concat([arsenal_test, moved_season])
arsenal_test.shape

(114, 25)

In [20]:
arsenal_test = arsenal_test.sort_values("date_time")
arsenal_test.to_csv("../feature_engineering_data/test/arsenal.csv", index=False)

In [21]:
# xoá các mùa giải đã chuyển sang tập test
arsenal_train = arsenal_train[~((arsenal_train["season"] == "2021-2022") | (arsenal_train["season"] == "2022-2023"))]
arsenal_train.shape

(799, 25)

In [22]:
arsenal_train.to_csv("../feature_engineering_data/train/arsenal.csv", index=False)

#### 2. Điều chỉnh lại tập crawl data từ trước, chia bớt data tập train qua bên tập test

In [4]:
crawl_train_df = pd.read_csv("../raw_data/raw_data_train.csv")
crawl_test_df = pd.read_csv("../raw_data/raw_data_test.csv")

moved_season = crawl_train_df[(crawl_train_df["season"] == "2021-2022") | (crawl_train_df["season"] == "2022-2023")]
moved_season.shape # 1520 = 2 * 20 * 38

(0, 17)

In [5]:
print("Before: ", crawl_test_df.shape)
crawl_test_df = pd.concat([crawl_test_df, moved_season])
print("After: ", crawl_test_df.shape)

Before:  (2280, 22)
After:  (2280, 22)


In [5]:
# Xoá các mùa giải đã chuyển sang tập test
print("Before: ", crawl_train_df.shape)
crawl_train_df = crawl_train_df[~((crawl_train_df["season"] == "2021-2022") | (crawl_train_df["season"] == "2022-2023"))]
print("After: ", crawl_train_df.shape)

Before:  (17480, 17)
After:  (15960, 17)


In [11]:
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%Y-%m-%d').strftime('%Y-%m-%d')
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%m/%d/%Y').strftime('%Y-%m-%d')
        except ValueError:
            return pd.NaT

In [ ]:
crawl_train_df['date'] = pd.to_datetime(crawl_train_df['date']).dt.strftime('%Y-%m-%d')
crawl_test_df['date'] = pd.to_datetime(crawl_test_df['date']).dt.strftime('%Y-%m-%d')

In [8]:
# Lưu lại dữ liệu
crawl_train_df.to_csv("../raw_data/raw_data_train.csv", index=False)
crawl_test_df.to_csv("../raw_data/raw_data_test.csv", index=False)